In [110]:
# imports

import pandas as pd

### DFCI_2014_PES study

In [111]:
# get data
#TODO import as objects from harvester


mut_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_mutations.txt', sep='\t')
study_meta = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/meta_study.txt', sep='\t')
patient_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_patient.txt', sep='\t')
sample_df = pd.read_csv('/Users/costellh/repos/metakb/hmc_notebooks/es_dfarber_broad_2014_extracted/es_dfarber_broad_2014/data_clinical_sample.txt', sep='\t')

In [112]:
# clean mutations data

# subset data
# remove empty columns
emp_col = mut_df.isnull().all()
mut_df.dropna(how='all', axis=1, inplace=True)

# subset for necessary columns
mut_df = mut_df.filter(['Hugo_Symbol',
                        'Chromosome',
                        'Start_Position', 
                        'End_Position',
                        'Consequence',
                        'Variant_Classification',
                        'Variant_Type',
                        'Reference_Allele',
                        'Tumor_Seq_Allele2',
                        'Tumor_Sample_Barcode',
                        'Sequence_Source',
                        'HGVSc',
                        'HGVSp',
                        'HGVSp_Short',
                        'Transcript_ID',
                        'RefSeq',
                        'Protein_position'
                       ], axis=1)


# construct Gnomad variant ID column
mut_df["Gnomad_Notation"] = subset_mut_df["Chromosome"].astype(str) + "-" + subset_mut_df["Start_Position"].astype(str) + "-" + subset_mut_df["Reference_Allele"] + "-" + subset_mut_df["Tumor_Seq_Allele2"]


# Rename 'PATIENT_ID' to 'PatientID' in your DataFrame
mut_df = mut_df.rename(columns={'Tumor_Sample_Barcode': 'SAMPLE_ID'})


In [113]:
# clean patient data
# print(patient_df.shape)
# print(patient_df.columns)

# subset data
#remove first 4 rows
patient_df.columns = patient_df.iloc[3]
patient_df = patient_df.iloc[4:].reset_index(drop=True)
# print(patient_df.head(10))
# print(patient_df.shape)
# print(patient_df.columns)

# # subset for necessary columns
patient_df = patient_df.filter(['PATIENT_ID',
                        'AGE',
                        'SEX', 
                        'ETHNICITY',
                        'Consequence'
                       ], axis=1)

In [114]:
# clean sample data

# print(sample_df.shape)
# print(sample_df.columns)
# print(sample_df.head())


# subset data
# remove first 4 rows
sample_df.columns = sample_df.iloc[3]
sample_df = sample_df.iloc[4:].reset_index(drop=True)
# print(sample_df.head(10))
# print(sample_df.shape)
# print(sample_df.columns)

# subset for necessary columns
sample_df = sample_df.filter(['PATIENT_ID',
                                     'SAMPLE_ID',
                                     'SAMPLE_CLASS',
                                     'ONCOTREE_CODE',
                                     'CANCER_TYPE',
                                     'CANCER_TYPE_DETAILED',
                                     'TMB_NONSYNONYMOUS'
                                    ], axis=1)




# print(subset_sample_df.columns)

In [137]:
# combine dataframes
# combine sample and patient dfs
combined_df = sample_df.merge(patient_df, on='PATIENT_ID', how='left')

# print(sample_df.columns)
# print(sample_df.shape)

# print(patient_df.columns)
# print(patient_df.shape)

# print(combined_df.columns)
# print(combined_df.shape)

#add mut_df
combined_df = combined_df.merge(mut_df, on='SAMPLE_ID', how='left')

# print(mut_df.columns)
# print(mut_df.shape)

# print(combined_df.columns)
# print(combined_df.shape)
# print(combined_df.tail())

# add column for study id
study_id = study_meta.iloc[0, 0]
study_id = study_id.replace('cancer_study_identifier: ', '')
# study_id
combined_df['STUDY_ID'] = study_id
print(combined_df.head())

# remove duplicate variants per patient
# print(combined_df.shape)
combined_df = combined_df.drop_duplicates()
# print(combined_df.shape)
# removed 495 rows

# remove cell lines from data
combined_df = combined_df[combined_df['SAMPLE_CLASS'] != 'Cell line']
# print(combined_df.shape)
# removed 3123 rows



  PATIENT_ID SAMPLE_ID SAMPLE_CLASS ONCOTREE_CODE  CANCER_TYPE  \
0   SJDES004  SJDES004        Tumor            ES  Bone Cancer   
1   SJDES004  SJDES004        Tumor            ES  Bone Cancer   
2   SJDES004  SJDES004        Tumor            ES  Bone Cancer   
3   SJDES004  SJDES004        Tumor            ES  Bone Cancer   
4   SJDES004  SJDES004        Tumor            ES  Bone Cancer   

  CANCER_TYPE_DETAILED TMB_NONSYNONYMOUS AGE   SEX     ETHNICITY  ...  \
0        Ewing Sarcoma               0.3  15  Male  White/Europe  ...   
1        Ewing Sarcoma               0.3  15  Male  White/Europe  ...   
2        Ewing Sarcoma               0.3  15  Male  White/Europe  ...   
3        Ewing Sarcoma               0.3  15  Male  White/Europe  ...   
4        Ewing Sarcoma               0.3  15  Male  White/Europe  ...   

  Tumor_Seq_Allele2 Sequence_Source                         HGVSc  \
0                 C             WXS    ENST00000511912.1:c.364G>C   
1                 A       